# Low-Precision Analogue Inversion Sandbox

This notebook emulates the low-precision analogue inversion stage (A₀⁻¹) described in the HP-INV paper.
We model a 3-bit resistive array, inject analogue noise, and inspect how residuals evolve during
iterative refinement.


## Usage
- Adjust the matrix size, condition number, quantisation bits, and noise standard deviation in the parameters cell.
- Run the simulation cell to observe residual decay and error trajectories.
- Optional: Experiment with harsher noise or ill-conditioned matrices to stress-test convergence.


In [ ]:
import math
from dataclasses import dataclass
from typing import Tuple, List, Dict

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')


In [ ]:
# Simulation parameters
MATRIX_SIZE = 8
TARGET_CONDITION = 8.0  # aim for kappa(A) ≈ TARGET_CONDITION
ANALOG_BITS = 3
NOISE_STD = 1e-2  # gaussian noise added to analogue inversion output
MAX_ITERS = 10
SEED = 42

rng = np.random.default_rng(SEED)


In [ ]:
def generate_matrix(n: int, condition: float, rng: np.random.Generator) -> np.ndarray:
    """Create an n×n matrix with approximately the requested condition number."""
    u, _s, vt = np.linalg.svd(rng.normal(size=(n, n)))
    singulars = np.geomspace(condition, 1.0, num=n)
    s = np.diag(singulars)
    return u @ s @ vt

def quantise_symmetrically(a: np.ndarray, bits: int) -> Tuple[np.ndarray, float]:
    """Quantise matrix entries to symmetric levels using the provided bit-depth."""
    levels = 2 ** bits
    max_abs = np.max(np.abs(a))
    if max_abs == 0:
        return a.copy(), 1.0
    step = max_abs / (levels // 2 - 1)
    quantised = np.clip(np.round(a / step), -(levels // 2 - 1), levels // 2 - 1) * step
    return quantised, step

def analogue_lp_inverse(a_lp: np.ndarray, b: np.ndarray, noise_std: float, rng: np.random.Generator) -> np.ndarray:
    """Simulate the low-precision analogue inversion with additive gaussian noise."""
    x = np.linalg.solve(a_lp, b)
    noise = rng.normal(scale=noise_std, size=x.shape)
    return x + noise

def run_simulation(n: int, condition: float, bits: int, noise_std: float, max_iters: int, rng: np.random.Generator) -> Dict[str, List[float]]:
    a = generate_matrix(n, condition, rng)
    x_true = rng.normal(size=(n,))
    b = a @ x_true

    a_lp, step = quantise_symmetrically(a, bits)

    residuals = []
    errors = []
    x_est = np.zeros_like(x_true)
    r = b - a @ x_est

    for _ in range(max_iters):
        delta = analogue_lp_inverse(a_lp, r, noise_std, rng)
        x_est = x_est + delta
        r = b - a @ x_est

        residuals.append(np.linalg.norm(r))
        errors.append(np.linalg.norm(x_est - x_true) / np.linalg.norm(x_true))

    return {"A": a, "A_lp": a_lp, "step": step, "residuals": residuals, "errors": errors}


In [ ]:
results = run_simulation(
    n=MATRIX_SIZE,
    condition=TARGET_CONDITION,
    bits=ANALOG_BITS,
    noise_std=NOISE_STD,
    max_iters=MAX_ITERS,
    rng=rng,
)

iters = np.arange(1, MAX_ITERS + 1)
residuals_log2 = np.log2(results['residuals'])

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(iters, residuals_log2, marker='o')
axes[0].set_title('Residual Norm (log₂)')
axes[0].set_xlabel('Iteration')
axes[0].set_ylabel('log₂‖rₖ‖₂')

axes[1].plot(iters, results['errors'], marker='o', color='tab:orange')
axes[1].set_title('Relative Error vs. Iteration')
axes[1].set_xlabel('Iteration')
axes[1].set_ylabel('‖x̂ₖ − x*‖ / ‖x*‖')

fig.suptitle('Low-Precision Analogue Inversion Sandbox', fontsize=14)
plt.tight_layout()
plt.show()


## Inspect Quantised Matrix
The cell below compares the original matrix to its 3-bit analogue representation. Large quantisation steps
illustrate the limited precision that the iterative refinement must overcome.


In [ ]:
import pandas as pd

def format_matrix(mat: np.ndarray) -> pd.DataFrame:
    return pd.DataFrame(mat).round(4)

pd.concat(
    {
        'A (ideal)': format_matrix(results['A']),
        f'A_lp ({ANALOG_BITS}-bit)': format_matrix(results['A_lp']),
    },
    axis=1
)


## Experiments to Try
- Increase `TARGET_CONDITION` to 50 or 100 and observe how convergence slows.
- Raise `NOISE_STD` to emulate driftier analogue hardware.
- Change `ANALOG_BITS` to 2 or 4 to explore quantisation trade-offs.
- Inject structured noise (e.g., bias) inside `analogue_lp_inverse` to mimic systematic offsets.
